In [23]:
import pandas as pd

In [24]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_30.csv')

In [25]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [26]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [27]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [28]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [29]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [30]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,ARI,6/30/2021,1,668942.0,Josh Rojas,1,Y,LF
1,ARI,6/30/2021,1,641796.0,Tim Locastro,2,Y,CF
2,ARI,6/30/2021,1,500871.0,Eduardo Escobar,3,Y,2B
3,ARI,6/30/2021,1,572233.0,Christian Walker,4,Y,1B
4,ARI,6/30/2021,1,452678.0,Asdrubal Cabrera,5,Y,3B
...,...,...,...,...,...,...,...,...
281,OAK,6/30/2021,1,476704.0,Jed Lowrie,6,N,NaN
282,OAK,6/30/2021,1,462101.0,Elvis Andrus,7,N,NaN
283,OAK,6/30/2021,1,621450.0,Skye Bolt,8,N,NaN
284,OAK,6/30/2021,1,621450.0,Skye Bolt,9,N,NaN


In [31]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [32]:
pitch_df.drop(456, inplace = True)

In [33]:
matchups

{'STL': 'ARI',
 'ARI': 'STL',
 'MIL': 'CHC',
 'CHC': 'MIL',
 'COL': 'PIT',
 'PIT': 'COL',
 'WAS': 'TB',
 'TB': 'WAS',
 'CLE': 'DET',
 'DET': 'CLE',
 'NYY': 'LAA',
 'LAA': 'NYY',
 'PHI': 'MIA',
 'MIA': 'PHI',
 'TOR': 'SEA',
 'SEA': 'TOR',
 'CIN': 'SD',
 'SD': 'CIN',
 'BOS': 'KC',
 'KC': 'BOS',
 'ATL': 'NYM',
 'NYM': 'ATL',
 'HOU': 'BAL',
 'BAL': 'HOU',
 'CHW': 'MIN',
 'MIN': 'CHW',
 'OAK': 'TEX',
 'TEX': 'OAK'}

In [34]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-34-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-34-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [35]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-35-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [36]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
9,STL,6/30/2021,1,666185.0,Dylan Carlson,1,Y,CF,R,Riley Smith,...,0.231760,0.309160,0.369099,0.678259,0.137339,0.308176,27.873883,-3.611760,0.297408,89.481313
10,STL,6/30/2021,1,502671.0,Paul Goldschmidt,2,Y,1B,R,Riley Smith,...,0.280864,0.358904,0.435185,0.794089,0.154321,0.337500,54.618967,10.212394,0.348034,122.352497
11,STL,6/30/2021,1,571448.0,Nolan Arenado,3,Y,3B,R,Riley Smith,...,0.263699,0.319876,0.486301,0.806177,0.222603,0.256098,44.620267,5.781619,0.336688,107.158994
12,STL,6/30/2021,1,641933.0,Tyler O'Neill,4,Y,LF,R,Riley Smith,...,0.212871,0.274775,0.480198,0.754973,0.267327,0.223140,27.710981,0.692501,0.320407,103.784961
13,STL,6/30/2021,1,572761.0,Matt Carpenter,5,Y,2B,R,Riley Smith,...,0.168421,0.307018,0.305263,0.612281,0.136842,0.215517,21.128179,-6.741757,0.279864,76.947232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,OAK,6/30/2021,1,640461.0,Chad Pinder,5,N,NaN,L,Kolby Allard,...,0.205128,0.261905,0.487179,0.749084,0.282051,0.192308,5.050016,-0.079217,0.315958,104.357008
281,OAK,6/30/2021,1,476704.0,Jed Lowrie,6,N,NaN,L,Kolby Allard,...,0.311688,0.345679,0.389610,0.735289,0.077922,0.393443,10.395922,0.951859,0.324083,112.399894
282,OAK,6/30/2021,1,462101.0,Elvis Andrus,7,N,NaN,L,Kolby Allard,...,0.155556,0.180851,0.233333,0.414184,0.077778,0.183099,0.777009,-10.495494,0.175478,8.813045
283,OAK,6/30/2021,1,621450.0,Skye Bolt,8,N,NaN,L,Kolby Allard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [38]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [39]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [40]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
9,STL,6/30/2021,1,666185.0,Dylan Carlson,1,Y,CF,R,Riley Smith,...,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789
10,STL,6/30/2021,1,502671.0,Paul Goldschmidt,2,Y,1B,R,Riley Smith,...,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000
11,STL,6/30/2021,1,571448.0,Nolan Arenado,3,Y,3B,R,Riley Smith,...,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000
12,STL,6/30/2021,1,641933.0,Tyler O'Neill,4,Y,LF,R,Riley Smith,...,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000
13,STL,6/30/2021,1,572761.0,Matt Carpenter,5,Y,2B,R,Riley Smith,...,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789,6.464789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,OAK,6/30/2021,1,640461.0,Chad Pinder,5,N,NaN,L,Kolby Allard,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
281,OAK,6/30/2021,1,476704.0,Jed Lowrie,6,N,NaN,L,Kolby Allard,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
282,OAK,6/30/2021,1,462101.0,Elvis Andrus,7,N,NaN,L,Kolby Allard,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
283,OAK,6/30/2021,1,621450.0,Skye Bolt,8,N,NaN,L,Kolby Allard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [42]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [43]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,ARI,6/30/2021,1,642092.0,Riley Smith,SP,Y,NaN,3,5,...,2.454545,0.691824,0.408805,0.283019,1.506849,7.772727,1.721312,3.597786,18.639635,23.912326
26,CHC,6/30/2021,1,453562.0,Jake Arrieta,SP,Y,NaN,3,5,...,6.654930,0.833333,0.353535,0.479798,1.366120,6.760563,1.503497,3.804348,23.218759,22.085876
44,PIT,6/30/2021,1,641771.0,Chad Kuhl,SP,Y,NaN,4,4,...,6.000000,0.748201,0.546763,0.201439,0.892857,5.866667,0.952381,2.476190,24.022439,22.078267
53,COL,6/30/2021,1,592351.0,Jon Gray,SP,Y,NaN,5,3,...,2.571429,0.620112,0.234637,0.385475,0.763975,3.928571,0.902439,2.142857,11.111999,12.396640
62,TB,6/30/2021,1,656876.0,Drew Rasmussen,SP,Y,NaN,3,5,...,6.750000,1.918605,0.639535,1.279070,1.000000,6.500000,1.538462,3.669725,17.300064,15.242671
71,WAS,6/30/2021,1,452657.0,Jon Lester,SP,Y,NaN,2,6,...,10.429185,0.906977,0.436047,0.470930,1.657143,8.652361,1.734940,4.659574,32.563848,31.511469
81,DET,6/30/2021,1,570632.0,Jose Urena,SP,Y,NaN,3,6,...,4.872180,1.325967,0.696133,0.629834,1.139241,6.902256,1.368421,4.303797,24.495792,27.091885
91,CLE,6/30/2021,1,615698.0,Cal Quantrill,SP,Y,NaN,5,4,...,5.142857,0.617450,0.295302,0.322148,1.138462,5.485714,1.207547,3.555556,19.973309,19.167488
111,NYY,6/30/2021,1,593334.0,Domingo German,SP,N,NaN,2,7,...,12.886364,1.810345,0.301724,1.508621,1.400000,6.443182,1.513514,6.428571,30.553963,26.519422
120,MIA,6/30/2021,1,656548.0,Jordan Holloway,SP,N,NaN,3,5,...,0.000000,1.071429,0.833333,0.238095,0.857143,6.500000,1.153846,4.615385,17.212369,25.067228


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [44]:
scores = pd.read_csv('../datasets/6_30_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210630,125r,605135,"Bassitt, Chris",1.0,0,1.0,31.35,10500.0,OAK,v tex,NaN,3,1
1,20210630,155a,656302,"Cease, Dylan",1.0,0,1.0,23.90,9700.0,CHW,v min,NaN,13,3
2,20210630,108m,503449,"Peralta, Wily",1.0,0,1.0,22.85,5100.0,DET,@ cle,2.0,7,1(F7)
3,20210630,143n,608331,"Fried, Max",1.0,9,1.0,22.25,8300.0,ATL,v nym,NaN,20,2
4,20210630,130k,592351,"Gray, Jon",1.0,9,1.0,19.30,7900.0,COL,v pit,NaN,6,2


In [45]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [46]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [47]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [48]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [49]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [50]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [51]:
bat_final.to_csv('../merged_data/hitters_6_30.csv')
pitch_final.to_csv('../merged_data/pitchers_6_30.csv')

In [52]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,STL,6/30/2021,1,666185.0,Dylan Carlson,1,Y,CF,R,Riley Smith,...,1.0,1,7.0,3.0,3600.0,STL,v ari,NaN,7,4
1,STL,6/30/2021,1,502671.0,Paul Goldschmidt,2,Y,1B,R,Riley Smith,...,1.0,2,3.0,17.0,4200.0,STL,v ari,NaN,7,4
2,STL,6/30/2021,1,571448.0,Nolan Arenado,3,Y,3B,R,Riley Smith,...,1.0,3,5.0,0.0,5700.0,STL,v ari,NaN,7,4
3,STL,6/30/2021,1,641933.0,Tyler O'Neill,4,Y,LF,R,Riley Smith,...,1.0,4,7.0,26.0,3800.0,STL,v ari,NaN,7,4
4,STL,6/30/2021,1,572761.0,Matt Carpenter,5,Y,2B,R,Riley Smith,...,1.0,5,34.0,0.0,3300.0,STL,v ari,NaN,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,OAK,6/30/2021,1,621566.0,Matt Olson,3,N,NaN,L,Kolby Allard,...,1.0,3,3.0,0.0,6000.0,OAK,v tex,NaN,3,1
193,OAK,6/30/2021,1,657656.0,Ramon Laureano,4,N,NaN,L,Kolby Allard,...,1.0,1,7.0,0.0,5400.0,OAK,v tex,NaN,3,1
194,OAK,6/30/2021,1,640461.0,Chad Pinder,5,N,NaN,L,Kolby Allard,...,1.0,5,7.0,5.0,3300.0,OAK,v tex,NaN,3,1
195,OAK,6/30/2021,1,476704.0,Jed Lowrie,6,N,NaN,L,Kolby Allard,...,1.0,6,4.0,0.0,4200.0,OAK,v tex,NaN,3,1


In [53]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,ARI,6/30/2021,1,642092.0,Riley Smith,SP,Y,NaN,3,5,...,1.0,9,1.0,-3.45,7000.0,ARI,@ stl,NaN,4,7
1,CHC,6/30/2021,1,453562.0,Jake Arrieta,SP,Y,NaN,3,5,...,1.0,9,1.0,-3.05,8300.0,CHC,@ mil,NaN,7,15
2,PIT,6/30/2021,1,641771.0,Chad Kuhl,SP,Y,NaN,4,4,...,1.0,9,1.0,10.35,6200.0,PIT,@ col,NaN,2,6
3,COL,6/30/2021,1,592351.0,Jon Gray,SP,Y,NaN,5,3,...,1.0,9,1.0,19.30,7900.0,COL,v pit,NaN,6,2
4,TB,6/30/2021,1,656876.0,Drew Rasmussen,SP,Y,NaN,3,5,...,1.0,9,1.0,2.70,4000.0,TAM,@ was,NaN,6,15
5,WAS,6/30/2021,1,452657.0,Jon Lester,SP,Y,NaN,2,6,...,1.0,9,1.0,5.85,8900.0,WAS,v tam,NaN,15,6
6,DET,6/30/2021,1,570632.0,Jose Urena,SP,Y,NaN,3,6,...,1.0,0,1.0,5.90,5100.0,DET,@ cle,NaN,9,4(F7)
7,CLE,6/30/2021,1,615698.0,Cal Quantrill,SP,Y,NaN,5,4,...,1.0,0,1.0,-0.50,6800.0,CLE,v det,NaN,4,9(F7)
8,NYY,6/30/2021,1,593334.0,Domingo German,SP,N,NaN,2,7,...,1.0,0,1.0,0.55,8500.0,NYY,v laa,NaN,8,11
9,MIA,6/30/2021,1,656548.0,Jordan Holloway,SP,N,NaN,3,5,...,1.0,9,1.0,1.35,4400.0,MIA,@ phi,NaN,11,6
